In [16]:
import numpy as np
import pandas as pd

from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing
from sklearn.decomposition import PCA

import sys
sys.path.append('./../../')

from src.test_algorithms.err_relativo import err_relativo

# Método de la Potencia

En muchas aplicaciones del mundo real de la ciencia y la ingeniería, se requiere encontrar numéricamente el valor Eigen más grande o dominante y el Eigenvector correspondiente. Existen diferentes como el método de la potencia que sigue un enfoque iterativo y conforma un algoritmo simple.

El siguiente algoritmo busca desarrollar los pasos necesario para el método de la potencia y así después pues ser implementado como otro método alternativo a los demás que se tienen en el proyecto.

Dada una matriz $A$ diagonalizable, el algoritmo de la potencia genera un número $\lambda$ que es el eigenvalor más grande (en valor asoluto) de $A$, y un vector no nulo $v$, que es el eigenvector correspondiente a $\lambda$, es decir, son tales que $Av = \lambda v$.

El método de la potencia comienza con un vector $b_0$, que puede ser un vector aleatoio, o bien una aproximación al eigenvector dominante. La relación de recurrencia que describe al método es:
$$b_{k+1}= \frac{Ab_k}{\|Ab_k\|}$$
Por lo que, en cada iteración, el vector $b_k$ es multiplicado por la matriz $A$ y luego normalizado.

Buscamos una sucesión $(b_k)$ que converja a un eigenvector Para asegurar la convergencia, necesitan cumplirse las siguientes condiciones:

- $A$ tiene un eigenvalor estrictamente mayor en magnitud respecto a sus otros eigenvalores
- El vector de inicio $b_0$ tiene una componente distinta de cero en la dirección de un eigenvector asociado con el eigenvalor dominante.

Además, bajo las dos supoosiciones anteriores, la sucesión $(\mu_k)$ definida por:
$$\mu_k = \frac{b_k^TAb_k}{b_k^Tb_k}$$
converge al eigenvalor dominante. 


In [17]:
df = pd.read_csv('../../data/nndb_flat.csv', encoding = "UTF-8")

In [18]:
df.drop(df.columns[df.columns.str.contains('_USRDA')].values, 
        inplace=True, axis=1)
df = df.drop(columns=['ID','FoodGroup','ShortDescrip','Descrip','CommonName','MfgName','ScientificName'])

In [19]:
####Se estandarizan los datos.
scaler = preprocessing.StandardScaler()
X = scaler.fit_transform(df)

In [20]:
X

array([[ 2.89623357, -1.01174721,  4.44128945, ..., -0.64991809,
        -0.41055694, -0.55991833],
       [ 2.89623357, -1.01174721,  4.44128945, ..., -0.65484222,
        -0.41055694, -0.57183012],
       [ 3.83495634, -1.06577576,  5.59915265, ..., -0.75332487,
        -0.44590424, -0.58374191],
       ...,
       [ 0.25127886, -1.0923161 , -0.67108312, ..., -0.72870421,
        -0.42116113, -0.53013886],
       [-0.80552224,  0.43375349, -0.58284096, ...,  0.57126681,
         0.52261173, -0.28892514],
       [-0.81142615,  0.78446513, -0.63956806, ...,  0.11824661,
         0.14793037, -0.28892514]])

In [21]:
C = np.dot(X.T, X)/X.shape[0]

print('Matriz de Covarianzas')
print(C)

Matriz de Covarianzas
[[ 1.00000000e+00  1.10302026e-01  8.06794721e-01  4.87096292e-01
   3.13151419e-01  1.97337706e-01  2.63288685e-02  1.21872046e-01
  -1.22922092e-02 -3.34948126e-02  3.03701016e-01  1.45936327e-01
   1.74991200e-01  1.55830467e-01  1.87438795e-01  1.23571027e-01
   9.95128814e-02  1.95564305e-01  2.55879578e-01  3.76210142e-02
   1.95180646e-01  5.66727437e-02  1.12766353e-01]
 [ 1.10302026e-01  1.00000000e+00  5.47153843e-02 -3.01973931e-01
  -2.66169540e-01 -7.27774043e-02  2.64827958e-02  2.28485319e-01
   2.45423401e-01 -6.64555794e-02 -2.94814592e-02  8.93927218e-03
   3.76939299e-01  2.02479732e-01  9.83597137e-02  4.68882866e-02
   1.58899609e-01  1.26375330e-01  2.18610589e-01  3.97424386e-02
   4.44607605e-01  3.78369173e-01  4.16315378e-01]
 [ 8.06794721e-01  5.47153843e-02  1.00000000e+00 -5.41161330e-02
  -1.96205883e-03 -2.88420113e-02  2.43786763e-02 -4.69495534e-02
  -2.07856488e-02 -5.99934668e-02  3.38008279e-01 -5.80492134e-02
  -2.26710652e-02 

Definimos la función que implementa el método de la potencia para obtener el eigenvalor dominante y su correspondiente eigenvector:

In [22]:
def power_iteration(A, num_simulations: int):
    # Ideally choose a random vector
    # To decrease the chance that our vector
    # Is orthogonal to the eigenvector
    b_k = np.random.rand(A.shape[1])

    for _ in range(num_simulations):
        # calculate the matrix-by-vector product Ab
        b_k1 = np.dot(A, b_k)

        # calculate the norm
        b_k1_norm = np.linalg.norm(b_k1)

        # re normalize the vector
        b_k = b_k1 / b_k1_norm
    
    #Obtenemos el eigenvalor correspondiente a b_k con el cociente de Rayleigh
    m_k = (b_k.T@A@b_k)/(b_k.T@b_k)
    
    #Devolvemos el mayor eigenvalor y su correspondiente eigenvector
    return m_k,b_k
    

In [23]:
m_1,b_1 = power_iteration(C,500)

In [24]:
#eigenvalor dominante
m_1

5.449285764311568

In [25]:
#eigenvector correspondiente al eigenvalor dominante
b_1

array([0.15781363, 0.1406199 , 0.03300812, 0.16968451, 0.07632275,
       0.18156988, 0.13351884, 0.31566282, 0.17798456, 0.08763937,
       0.13712201, 0.28410233, 0.33777895, 0.34132464, 0.27245318,
       0.16811197, 0.18080591, 0.2998569 , 0.24134847, 0.09356749,
       0.19940258, 0.0923186 , 0.24355123])

Podemos comparar los valores obtenidos con el primer eigenvalor y eigenvector:

In [26]:
evalues, evectors = np.linalg.eig(C)

In [27]:
evalues[0]

5.449285764311579

In [28]:
evectors.T[0]

array([-0.15781363, -0.1406199 , -0.03300812, -0.16968451, -0.07632275,
       -0.18156988, -0.13351884, -0.31566282, -0.17798456, -0.08763937,
       -0.13712201, -0.28410233, -0.33777895, -0.34132464, -0.27245318,
       -0.16811197, -0.18080591, -0.2998569 , -0.24134847, -0.09356749,
       -0.19940258, -0.0923186 , -0.24355123])

In [29]:
np.allclose(m_1,evalues[0])

True

In [30]:
np.allclose(np.abs(b_1),np.abs(evectors.T[0]))

True

## Deflation

Con lo anterior, hemos obtenido el eigenvalor de mayor magnitud y su correspondiente eigenvalor. Sin embargo, para hacer el PCA necesitamos los demás eigenvalores. Es aquí donde entra el método de *deflation*. Este consiste en volver a aplicar el método a una matriz actualizada:
$$A_{k+1}= A_k - b_kb_k^TA_kb_kb_k^T$$

Apliquemos dicha transformación a la matriz de covarianzas $C$ para obtener el segundo eigenvalor de $C$ y su correspondiente eigenvector:

In [31]:
C_def = C- np.outer(b_1,b_1)@C@np.outer(b_1,b_1)

In [32]:
m_2,b_2 = power_iteration(C_def,100)

In [33]:
m_2

2.6184583526663694

In [34]:
b_2

array([ 0.27344866, -0.34339677,  0.11167035,  0.44341644,  0.35876913,
        0.25773287, -0.23647025, -0.02112889, -0.35504461,  0.03852455,
        0.10637166,  0.09709328, -0.08480073, -0.07347102,  0.0751503 ,
        0.10517259, -0.21266898,  0.09381195,  0.10336119, -0.08878313,
       -0.08744763, -0.23932243, -0.17779772])

Podemos corroborar que estos valores aproximan el segundo eigenvalor y su correspondiente eigenvector

In [35]:
evalues[1]

2.6184583526663796

In [36]:
evectors.T[1]

array([-0.27344866,  0.34339677, -0.11167035, -0.44341644, -0.35876913,
       -0.25773287,  0.23647025,  0.02112889,  0.35504461, -0.03852455,
       -0.10637166, -0.09709328,  0.08480073,  0.07347102, -0.0751503 ,
       -0.10517259,  0.21266898, -0.09381195, -0.10336119,  0.08878313,
        0.08744763,  0.23932243,  0.17779772])

Así, creamos una función que combine el método de la potencia y *deflation*:


In [37]:
def power_deflation(A,iter):
    #numero de columnas
    n = A.shape[1]
    # Inicializamos arrays de ceros
    eigenvalues = np.zeros(n)
    eigenvectors = np.zeros((n,n))
    #Hago una copia de la matriz original
    A_def = A.copy()
    #Iteramos tantas veces como columnas de la matriz
    for i in range(n):
        #Aplicamos el método de la potencia
        m_def,b_def = power_iteration(A_def,iter)
        #Actualizamos los arrays de eigen valores y vectores
        eigenvalues[i] = m_def
        eigenvectors[:,i]= b_def
        # Matriz actualizada
        A_def = A_def - np.outer(b_def,b_def)@A_def@np.outer(b_def,b_def)
    return eigenvalues, eigenvectors

In [38]:
evalues_pow, evectors_pow = power_deflation(C,50)

Podemos notar que (salvo por el orden decreciente en que nosotros obtenemos los valores), se trata de una buena aproximación:

In [39]:
# Lo que obtenemos
evalues_pow

array([5.44928576e+00, 2.61845828e+00, 2.03176806e+00, 1.87927007e+00,
       1.63567314e+00, 1.14019627e+00, 1.06104466e+00, 9.26240016e-01,
       8.61270808e-01, 8.25488584e-01, 7.31240120e-01, 5.96894949e-01,
       5.08500550e-01, 4.69626863e-01, 4.07986317e-01, 3.35848784e-01,
       3.29801786e-01, 3.23180047e-01, 2.56191219e-01, 2.38151299e-01,
       2.11158953e-01, 1.59093679e-01, 3.79524723e-03])

In [40]:
#Lo que esperamos (ordenado de forma decreciente)
evalues = np.sort(evalues)[::-1]
evalues

array([5.44928576e+00, 2.61845835e+00, 2.03189759e+00, 1.87913144e+00,
       1.63567140e+00, 1.14037066e+00, 1.06086727e+00, 9.26253795e-01,
       8.62018168e-01, 8.24686441e-01, 7.31232556e-01, 5.96894964e-01,
       5.08653057e-01, 4.69461661e-01, 4.07986316e-01, 3.37934003e-01,
       3.29568690e-01, 3.21238326e-01, 2.56202403e-01, 2.38139303e-01,
       2.11158912e-01, 1.59093679e-01, 3.79524723e-03])

## PCA

In [41]:
###  devuelve el comp-ésimo eigenvalor y su eigenvector
def calPCA(sigma, comp):
    eig_vals, eig_vecs = power_deflation(sigma,50)
    return {'eig_val': eig_vals[comp-1], 'eig_vec': eig_vecs[:,comp-1]}

In [42]:
total_var = 0
for i in range(X.shape[1]):
    total_var = total_var + calPCA(C, i+1)['eig_val']
total_var

22.996844165965722

In [43]:
np.sum(evalues_pow)

23.00016547471999

In [46]:
def PCA_from_potencia(X):
    prop = 0 #Proporción de varianza explicada
    comp = 1 
    cur_var = 0
    comp_vecs = np.zeros([X.shape[1], X.shape[1]])
    
    # convertir a array
    A = np.array(X)
    
    # Centrar los datos
    mean_vec = np.mean(A, axis=0)
    datos_centrados = (A - mean_vec)
    
    #Calculamos la matriz de covarianzas
    cov = np.dot(X.T, X)/X.shape[0]
    
    #Aplicamos el método de la potencia
    evalues_pow, evectors_pow = power_deflation(cov,50)
    
    # La varianza explicada
    varianza_explicada = evalues_pow/np.sum(evalues_pow)
    
    # Los datos transformados (componentes principales)
    Z = datos_centrados@evectors_pow
    
    
    # Calcula número de componentes de manera automatica de acuerdo a la variana explicada
    # Threshold de 80%
    n = X.shape[1] #numero de columnas
    varianza_acumulada = varianza_explicada.cumsum()
    conteo = (varianza_acumulada)  <  0.8
    num_componentes = conteo.sum() + 1
    
    return evalues_pow[:num_componentes], evectors_pow[:num_componentes], Z[:,:num_componentes], varianza_explicada[:num_componentes] 
    

In [47]:
PCA_from_potencia(X)

(array([5.44928576, 2.61845835, 2.03189438, 1.87913487, 1.63567144,
        1.14036142, 1.06087647, 0.92445864, 0.8638228 , 0.82455041]),
 array([[ 1.57813630e-01,  2.73451306e-01,  4.62239418e-01,
          5.00964102e-02,  2.93574522e-01,  1.33579181e-01,
          6.50994393e-02,  7.88950351e-02,  1.96305449e-02,
         -1.35810647e-01, -1.48565131e-01, -1.17149616e-01,
          6.32426439e-02,  6.14466536e-02, -3.88280756e-02,
         -3.35393990e-02,  2.59279970e-02, -1.20147047e-01,
         -7.84001586e-02,  7.57914501e-02, -1.29365504e-01,
          3.80913949e-02,  6.78011231e-01],
        [ 1.40619902e-01, -3.43395589e-01,  2.12142579e-01,
         -3.12170339e-01, -1.30490950e-02,  1.41530242e-01,
          4.36727119e-03,  1.39221514e-01,  1.24052861e-01,
         -1.10511415e-02, -3.08171327e-01,  1.29483795e-01,
          3.02441130e-01, -1.15138485e-01,  1.31510901e-01,
          1.12573113e-01,  4.69611544e-01,  1.82747179e-01,
         -3.84792463e-02,  2.26149730e

## Referencias
- Wikipedia [Power iteration](https://en.wikipedia.org/w/index.php?title=Power_iteration&oldid=957783806) (last visited May 29, 2020)
- Mackey, Lester. (2008). Deflation Methods for Sparse PCA. Advances in Neural Information Processing Systems 21 - Proceedings of the 2008 Conference. 21. 1017-1024. 
- Power Method Algorithm for Finding Dominant Eigen Value and Eigen Vector. (n.d.). Retrieved May 23, 2020, from https://www.codesansar.com/numerical-methods/power-method-algorithm-for-finding-dominant-eigen-value-and-eigen-vector.htm

- Fox, J., Chalmers, P., Monette, G., & Sanchez, G. (2020, April 14). powerMethod: Power Method for Eigenvectors in matlib: Matrix Functions for Teaching and Learning Linear Algebra and Multivariate Statistics. Retrieved from https://rdrr.io/cran/matlib/man/powerMethod.html

- Dan, D. J. (n.d.). dianejdan/Power-Method-PCA. Retrieved May 27, 2020, from https://github.com/dianejdan/Power-Method-PCA/blob/master/power-pca.py